In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
yeomi_review = db['yeomi_review']
yeomi_review = yeomi_review.find()
yeomi_review = pd.DataFrame(yeomi_review)
kto9suk9suk_review = db['kto9suk9suk_review']
kto9suk9suk_review = kto9suk9suk_review.find()
kto9suk9suk_review = pd.DataFrame(kto9suk9suk_review)

In [3]:
train_data1 = pd.read_excel("text_labeling_Validation.xlsx")
train_data2 = pd.read_excel("text_labeling_Training.xlsx")
train_data=pd.concat(objs=[train_data1,train_data2])

In [4]:
# 텍스트 파일 경로
file_path = 'ratings.txt'

# 텍스트 파일 읽기
with open(file_path, 'r') as file:
    text_data = file.read()

In [5]:
import pandas as pd
from io import StringIO

# 텍스트 데이터를 파일 객체로 변환
file_obj = StringIO(text_data)

# 데이터프레임 생성
train_data = pd.read_csv(file_obj, sep='\t')

In [6]:
train_data

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


# 데이터 전처리

## 여행에 미치다 페이지에서 국내 여행 게시글만 가져오기

In [7]:
list_region = ['#강원','#경기','#경남','#경북','#광주','#대구','#대전','#부산','#서울','#세종','#울산','#인천','#전남','#전북','#제주','#충남','#충북','#강릉','#고성','#동해','#삼척','#속초','#양구','#양양','#영월','#원주','#인제','#정선','#철원','#춘천','#태백','#평창','#홍천','#화천','#횡성','#가평','#고양','#과천','#광명','#광주','#구리','#군포','#김포','#남양주','#동두천','#부천','#성남','#수원','#시흥','#안산','#안성','#안양','#양주','#양평','#여주','#연천','#오산','#용인','#의왕','#의정부','#이천','#파주','#평택','#포천','#하남','#화성','#거제','#거창','#고성','#김해','#남해','#밀양','#사천','#산청','#양산','#의령','#진주','#창녕','#창원','#통영','#하동','#함안','#함양','#합천','#경산','#경주','#고령','#구미','#김천','#문경','#봉화','#상주','#성주','#안동','#영덕','#영양','#영주','#영천','#예천','#울릉','#울진','#의성','#청도','#청송','#칠곡','#포항','#광산','#남구','#동구','#북구','#서구','#군위','#남구','#달서','#달성','#동구','#북구','#서구','#수성','#중구','#대덕','#동구','#서구','#유성','#중구','#강서','#금정','#기장','#남구','#동구','#동래','#부산진','#북구','#사상','#사하','#서구','#수영','#연제','#영도','#중구','#해운대','#강남','#강동','#강북','#강서','#관악','#광진','#구로','#금천','#노원','#도봉','#동대문','#동작','#마포','#서대문','#서초','#성동','#성북','#송파','#양천','#영등포','#용산','#은평','#종로','#중구','#중랑','#남구','#동구','#북구','#울주','#중구','#강화','#계양','#남동','#동구','#미추홀','#부평','#서구','#연수','#옹진','#중구','#강진','#고흥','#곡성','#광양','#구례','#나주시','#담양','#목포','#무안','#보성','#순천','#신안','#여수','#영광','#영암','#완도','#장성','#장흥','#진도','#함평','#해남','#화순','#고창','#군산','#김제','#남원','#무주','#부안','#순창','#완주','#익산','#임실','#장수','#전주','#정읍','#진안','#서귀포','#제주','#계룡','#공주','#금산','#논산','#당진','#보령','#부여','#서산','#서천','#아산','#예산','#천안','#청양','#태안','#홍성','#괴산','#단양','#보은','#영동','#옥천','#음성','#제천','#증평','#진천','#청주','#충주']
korea_trip = []
for i in yeomi_review[yeomi_review['review_content'].isna()]['card_content']:
    for j in list_region:
        if j in i:
            korea_trip.append(i)
            break
condition_region = f"card_content in {korea_trip}"
korea_yeomi_review = yeomi_review.query(condition_region)
korea_yeomi_review

,_id,card_content,card_date,card_like,card_watch,check_point,review_content,review_date
201,65f9bf4f08bfcc46ca193606,".\n#인제, #Inje, #대한민국, #Korea\n.\n.\n자작나무 숲 분위기...",2018년 1월 22일,7447,,BePBx_qBqwm,NaN,NaN
202,65f9bf4f08bfcc46ca193607,".\n#인제, #Inje, #대한민국, #Korea\n.\n.\n자작나무 숲 분위기...",2018년 1월 22일,7447,,BePBx_qBqwm,@shorokhofff,2018년 6월 22일
203,65f9bf4f08bfcc46ca193608,".\n#인제, #Inje, #대한민국, #Korea\n.\n.\n자작나무 숲 분위기...",2018년 1월 22일,7447,,BePBx_qBqwm,@soye0114 여기 다니엘 뮤비 찍은곳인가???!!!,2018년 2월 24일
204,65f9bf5008bfcc46ca193609,".\n#인제, #Inje, #대한민국, #Korea\n.\n.\n자작나무 숲 분위기...",2018년 1월 22일,7447,,BePBx_qBqwm,@h.a.ing 요기봐 ㅋㅋㅋㅋㅋㅋ,2018년 2월 8일
205,65f9bf5008bfcc46ca19360a,".\n#인제, #Inje, #대한민국, #Korea\n.\n.\n자작나무 숲 분위기...",2018년 1월 22일,7447,,BePBx_qBqwm,@dal.hye 야 나 여기 가고싶어ㅠㅠㅠ,2018년 1월 28일
...,...,...,...,...,...,...,...,...
1152424,662b80e0f231019cd5b61943,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,C6OA2Fbhy5r,NaN,NaN
1152425,662b80e0f231019cd5b61944,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,C6OA2Fbhy5r,미츠바 좋습니당,2024년 4월 26일
1152426,662b80e0f231019cd5b61945,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,C6OA2Fbhy5r,와 이제 곧 장미공원의 시즌이다,2024년 4월 26일
1152427,662b80e1f231019cd5b61946,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,C6OA2Fbhy5r,혹시 여기 다 가본 사람 있어? 어디가 젤 좋았는지 추천해주라줘~🙋🏻‍♀️,2024년 4월 26일


## 구석구석 페이지와 여행에 미치다 페이지에서 가져온 데이터 합치기

In [8]:
kto9suk9suk_review

,_id,card_content,card_date,card_like,card_watch,check_point,review_content,review_date
0,65f99f8608bfcc46ca19351d,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,pkcKuIzTJ0,NaN,NaN
1,65f99f8608bfcc46ca19351e,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,pkcKuIzTJ0,정말 아름다워요,2014년 7월 9일
2,65f99f8608bfcc46ca19351f,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,pkcKuIzTJ0,멋진 섬이네요,2014년 6월 24일
3,65f99f8808bfcc46ca193520,"멋진 풍경과 감동, 여행과 휴식은 생각보다 가까운 곳에 있다. 부산 만덕 레고마을\...",2014년 6월 23일,87,,pk8YhwTTDR,NaN,NaN
4,65f99f8b08bfcc46ca193521,"유유자적 한가로이 풀을 뜯는 말 뒤편,\n걸려있는 구름마저도 평화로운 성산일출봉\n...",2014년 7월 10일,42,,qQx_BQTTBH,NaN,NaN
...,...,...,...,...,...,...,...,...
125933,662a1fb44a4fe5fadf3f71ee,신규 개장 기념 무료 운영 중(~5/6)‼ 짜릿함 최고인😆 김천 #사계절썰매장\n\...,2024년 4월 25일,106,,C6LYPKTSzun,🛷,2024년 4월 25일
125934,662f1d43b33c9dc30aa95f3f,"제주도 토종 소, 흑한우 살짝 레어로 익혀 한 입에 쏘옥~!\n참 숯불로 구워 더욱...",2014년 8월 5일,81,,rT9rIsTTNs,NaN,NaN
125935,662f1d4bb33c9dc30aa95f40,범선의 휘황찬란한 실루엣이 영덕 대게를 먹으러 찾는 방문객을 반기는 이곳 +_+ 대...,2014년 8월 26일,100,,sJhwZxzTCf,NaN,NaN
125936,662f22cb5cb025207273f6f6,"제주도 토종 소, 흑한우 살짝 레어로 익혀 한 입에 쏘옥~!\n참 숯불로 구워 더욱...",2014년 8월 5일,81,,rT9rIsTTNs,NaN,NaN


In [9]:
df_sns = pd.concat([kto9suk9suk_review, korea_yeomi_review], ignore_index=True)
df_sns

,_id,card_content,card_date,card_like,card_watch,check_point,review_content,review_date
0,65f99f8608bfcc46ca19351d,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,pkcKuIzTJ0,NaN,NaN
1,65f99f8608bfcc46ca19351e,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,pkcKuIzTJ0,정말 아름다워요,2014년 7월 9일
2,65f99f8608bfcc46ca19351f,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,pkcKuIzTJ0,멋진 섬이네요,2014년 6월 24일
3,65f99f8808bfcc46ca193520,"멋진 풍경과 감동, 여행과 휴식은 생각보다 가까운 곳에 있다. 부산 만덕 레고마을\...",2014년 6월 23일,87,,pk8YhwTTDR,NaN,NaN
4,65f99f8b08bfcc46ca193521,"유유자적 한가로이 풀을 뜯는 말 뒤편,\n걸려있는 구름마저도 평화로운 성산일출봉\n...",2014년 7월 10일,42,,qQx_BQTTBH,NaN,NaN
...,...,...,...,...,...,...,...,...
599153,662b80e0f231019cd5b61943,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,C6OA2Fbhy5r,NaN,NaN
599154,662b80e0f231019cd5b61944,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,C6OA2Fbhy5r,미츠바 좋습니당,2024년 4월 26일
599155,662b80e0f231019cd5b61945,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,C6OA2Fbhy5r,와 이제 곧 장미공원의 시즌이다,2024년 4월 26일
599156,662b80e1f231019cd5b61946,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,C6OA2Fbhy5r,혹시 여기 다 가본 사람 있어? 어디가 젤 좋았는지 추천해주라줘~🙋🏻‍♀️,2024년 4월 26일


## 어느 지역에 관한 표시

In [10]:
region_gw = ['강원도 ','강원 ','강원특별자치도 ','#강원','#강릉', '#고성', '#동해', '#삼척', '#속초', '#양구', '#양양', '#영월', '#원주', '#인제', '#정선', '#철원', '#춘천', '#태백', '#평창', '#홍천', '#화천', '#횡성']
region_gg = ['경기도 ','경기 ','#경기','#가평', '#고양', '#과천', '#광명', '#구리', '#군포', '#김포', '#남양주', '#동두천', '#부천', '#성남', '#수원', '#시흥', '#안산', '#안성', '#안양', '#양주', '#양평', '#여주', '#연천', '#오산', '#용인', '#의왕', '#의정부', '#이천', '#파주', '#평택', '#포천', '#하남', '#화성']
region_gn = ['경상남도 ','경남 ','#경남 ','#거제', '#거창', '#고성', '#김해', '#남해', '#밀양', '#사천', '#산청', '#양산', '#의령', '#진주', '#창녕', '#창원', '통영', '#하동', '#함안', '#함양', '#합천']
region_gb = ['경상북도 ','경북 ','#경북','#경산', '#경주', '#고령', '#구미', '#김천', '#문경', '#봉화', '#상주', '#성주', '#안동', '#영덕', '#영양', '#영주', '#영천', '#예천', '#울릉', '#울진', '#의성', '#청도', '#청송', '#칠곡', '#포항']
region_gj = ['광주광역시 ','광주 ','#광주','#광산']
region_dg = ['대구광역시 ','대구 ','#대구','#군위', '#달서', '#달성', '#수성']
region_dj = ['대전광역시 ','대전 ','#대전','#대덕',   '#유성']
region_sj = ['세종특별자치시 ','세종 ','#세종']
region_bs = ['부산특별시 ','부산 ','#부산', '#금정', '#기장',  '#동래', '#부산진', '#사상', '#사하',  '#수영', '#연제', '#영도',  '#해운대']
region_su = ['서울특별시 ','서울 ','#서울','#강남', '#강동', '#강북', '#관악', '#광진', '#구로', '#금천', '#노원', '#도봉', '#동대문', '#동작', '#마포', '#서대문', '#서초', '#성동', '#성북', '#송파', '#양천', '#영등포', '#용산', '#은평', '#종로',  '#중랑']
region_us = ['울산광역시 ', '울산 ','#울산', '#울주']
region_ic = ['인천광역시 ','인천 ','#인천','#강화', '#계양', '#남동',  '#미추홀', '#부평',  '#연수', '#옹진']
region_jn = ['전라남도 ','전남 ','#강진', '#전남','#고흥', '#곡성', '#광양', '#구례', '#나주', '#담양', '#목포', '#무안', '#보성', '#순천', '#신안', '#여수', '#영광', '#영암', '#완도', '#장성', '#장흥', '#진도', '#함평', '#해남', '#화순']
region_jb = ['전라북도 ','전북 ','#고창', '#전북','#군산', '#김제', '#남원', '#무주', '#부안', '#순창', '#완주', '#익산', '#임실', '#장수', '#전주', '#정읍', '#진안']
region_jj = ['제주특별자치도 ','제주 ','#서귀포', '#제주']
region_cn = ['충청남도 ','충남 ','#충남','#계룡', '#공주', '#금산', '#논산', '#당진', '#보령', '#부여', '#서산', '#서천', '#아산', '#예산', '#천안', '#청양', '#태안', '#홍성']
region_cb = ['충청북도 ','충북 ','#충북','#괴산', '#단양', '#보은', '#영동', '#옥천', '#음성', '#제천', '#증평', '#진천', '#청주', '#충주']

In [11]:
from collections import Counter

for i in range(len(df_sns)):
    region_tour=[]
    city_tour = []
    for j in region_gw:
        if j in df_sns['card_content'][i]:
            region_tour.append('강원')
            city_tour.append(j)
    for j in region_gg:
        if j in df_sns['card_content'][i]:
            region_tour.append('경기')
            city_tour.append(j)
    for j in region_gn:
        if j in df_sns['card_content'][i]:
            region_tour.append('경남')
            city_tour.append(j)
    for j in region_gj:
        if j in df_sns['card_content'][i]:
            region_tour.append('광주')   
            city_tour.append(j)
    for j in region_gb:
        if j in df_sns['card_content'][i]:
            region_tour.append('경북')   
            city_tour.append(j)
    for j in region_sj:
        if j in df_sns['card_content'][i]:
            region_tour.append('세종')   
            city_tour.append(j)
    for j in region_dg:
        if j in df_sns['card_content'][i]:
            region_tour.append('대구')
            city_tour.append(j)
    for j in region_dj:
        if j in df_sns['card_content'][i]:
            region_tour.append('대전')
            city_tour.append(j)
    for j in region_bs:
        if j in df_sns['card_content'][i]:
            region_tour.append('부산')
            city_tour.append(j)
    for j in region_su:
        if j in df_sns['card_content'][i]:
            region_tour.append('서울')
            city_tour.append(j)
    for j in region_us:
        if j in df_sns['card_content'][i]:
            region_tour.append('울산')
            city_tour.append(j)
    for j in region_ic:
        if j in df_sns['card_content'][i]:
            region_tour.append('인천')
            city_tour.append(j)
    for j in region_jn:
        if j in df_sns['card_content'][i]:
            region_tour.append('전남')    
            city_tour.append(j)
    for j in region_jb:
        if j in df_sns['card_content'][i]:
            region_tour.append('전북')
            city_tour.append(j)
    for j in region_jj:
        if j in df_sns['card_content'][i]:
            region_tour.append('제주')
            city_tour.append(j)
    for j in region_cn:
        if j in df_sns['card_content'][i]:
            region_tour.append('충남')
            city_tour.append(j)
    for j in region_cb:
        if j in df_sns['card_content'][i]:
            region_tour.append('충북')
            city_tour.append(j)
    word_region = Counter(region_tour)
    most_common_region = word_region.most_common(1)
    most_common_region_keys = [word for word, count in most_common_region]
    word_city = Counter(city_tour)
    most_common_city = word_city.most_common(1)
    most_common_city_keys = [word for word, count in most_common_city]
    df_sns.loc[i, 'region'] = ", ".join(most_common_region_keys)
    df_sns.loc[i, 'city'] = ", ".join(city_tour)


## 댓글에 관한 데이터와 제목에 관한 데이터 분류

In [12]:
df_sns_review = df_sns[df_sns['review_content'].notna()]
df_sns_title = df_sns[df_sns['review_content'].isna()]
df_sns_review = df_sns_review.reset_index()
df_sns_review
df_sns_title = df_sns_title.reset_index()
df_sns_title


,index,_id,card_content,card_date,card_like,card_watch,check_point,review_content,review_date,region,city
0,0,65f99f8608bfcc46ca19351d,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,pkcKuIzTJ0,NaN,NaN,,
1,3,65f99f8808bfcc46ca193520,"멋진 풍경과 감동, 여행과 휴식은 생각보다 가까운 곳에 있다. 부산 만덕 레고마을\...",2014년 6월 23일,87,,pk8YhwTTDR,NaN,NaN,부산,부산
2,4,65f99f8b08bfcc46ca193521,"유유자적 한가로이 풀을 뜯는 말 뒤편,\n걸려있는 구름마저도 평화로운 성산일출봉\n...",2014년 7월 10일,42,,qQx_BQTTBH,NaN,NaN,제주,#제주
3,7,65f99f8d08bfcc46ca193524,울산 명선도의 불게 타오르는 노을과 해무의 장관\n마치 그림을 그려놓은듯한 풍경에 ...,2014년 7월 11일,58,,qTGZrrzTAH,NaN,NaN,울산,"울산 , #울산"
4,11,65f99f8f08bfcc46ca193528,비가 내린 후에야 진정한 모습을 드러내는 시무지기폭포\n이 비경을 만나러 가는 고된...,2014년 7월 12일,39,,qVtXCrTTIY,NaN,NaN,광주,"광주 , #광주"
...,...,...,...,...,...,...,...,...,...,...,...
5187,599013,662abcaaf231019cd5b60515,"드디어 다음주, 국가권력급 벚꽃맛집 ‘여의도’ 봄꽃축제 개막 (뿌뿌-)\n\n누가 ...",2024년 3월 23일,3091,,C41qMeoBkpF,NaN,NaN,서울,"서울 , #서울"
5188,599038,662abe16f231019cd5b60b80,지금 제일 핫하다는 기안84 전시 ‘기안도(奇案島)’ 후기 🔥\n다들 여기서 기안8...,2024년 3월 26일,4752,,C49-AnfBEhj,NaN,NaN,서울,"서울 , #서울"
5189,599137,662ada15f231019cd5b61622,"지친 일상으로부터 벗어날 수 있는 시간, 응봉산에서 🌙\n\n이번 한주도 수고한 여...",2024년 4월 19일,1530,,C58BRwIhdW4,NaN,NaN,서울,"서울 , #서울"
5190,599140,662af284f231019cd5b61861,거창 여행 어떻게 할지 모르겠다고? 😎 여미가 직접 다녀와서 알려주는 [밥상의 여정...,2024년 4월 24일,1288,,C6I3IZBhfzp,NaN,NaN,경남,#거창


## 태그와 관련된 단어 제거

In [13]:
def split(text):
    clean_text = []
    for word in text.split():
        if word.startswith('@'):
            pass
        else:
            clean_text.append(word)
    return " ".join(clean_text)

In [14]:
df_sns_review.loc[:,'review_content'] = df_sns_review['review_content'].apply(split)
df_sns_review.query("review_content.str.contains('@')")

,index,_id,card_content,card_date,card_like,card_watch,check_point,review_content,review_date,region,city
390,459,65facb853d093f346325208f,경기도 #양평 세미원 에서 만난 싱그러운 초록 풍경! It's healing tim...,2014년 9월 9일,118,,stleS6TTAe,수경아 세미원이다!^@^ 우리가 걸어온 길...ㅎㅎ,2014년 9월 9일,경기,"경기도 , #양평"
578,690,65fbdbf9e60f7a9a412f0446,붉은빛과 푸른빛이 영롱하게 물 위로 쏟아져 내리는 여기는 대전 엑스포 다리! 견우직...,2014년 10월 23일,125,,ufCjFazTDs,ㅋㅋㅋㅋ #나도절절한사랑좀 ㅋㅋㅋㅋ@yoon.km,2014년 10월 23일,대전,"대전 , #대전"
673,811,65fbdc36e60f7a9a412f04bf,경산시장에서 만난 뜨끈하고 매콤한 어묵꼬치! 따뜻한 국물이 좋은걸 보니 겨울이 성큼...,2014년 11월 18일,145,,vh6aTcTTKr,아대박...@jammang_,2014년 11월 18일,경북,#경산
997,1230,65fd6a598cbcc379b4584772,길었던 연휴의 막바지. 새로운 한 주를 준비하며 여유로운 멋이 흐르는 충북 영동 월...,2015년 2월 22일,177,,zYx5axTTFt,저긴 아직도 미지의 세계.....ㅋㅋㅋㅋㅋㅋ@mis_7211,2015년 3월 23일,충북,"충북 , #영동"
1032,1275,65fe13048cbcc379b45855fa,갓 구워진 따끈따끈한 파이 냄새가 여기까지 전해지는 것 같지 않나요~? :) 이곳은...,2015년 3월 4일,135,,zzADJhTTC5,앗~!! 거두리.....@.@ 주말이 되면 거두리 카페거리에서 커피한잔 하고싶어지네...,2015년 3월 4일,강원,#춘천
...,...,...,...,...,...,...,...,...,...,...,...
591412,596575,662a3ea54a4fe5fadf3f78a5,“안 되는걸 되게 하면 되잖아요“ 그래서 우리는 눈오리 대신 흙오리를 만든답니다?\...,2024년 1월 19일,12.4만,,C2Q5szLhpdh,니@거덕,2024년 2월 22일,부산,"부산 , #부산"
591617,596780,662a3eb04a4fe5fadf3f7972,“안 되는걸 되게 하면 되잖아요“ 그래서 우리는 눈오리 대신 흙오리를 만든답니다?\...,2024년 1월 19일,12.4만,,C2Q5szLhpdh,아닠ㅋㅋㅋㅋㅋ@look_atme_mi,2024년 1월 19일,부산,"부산 , #부산"
592408,597581,662a4d9ef231019cd5b5d3fa,“내가 서울 좀 안다?” 주목👀 #광고\n나만 알고싶은 서울의 숨은 장소가 있다면?...,2024년 2월 8일,1934,,C3FYUTQBnWW,#관악구 #관악산연주대 가볍게 등산하기 너무 좋아요.❤️@jangyongseung4😍,2024년 2월 10일,서울,"#안산, 서울 , #서울, #서대문"
592685,597859,662a6853f231019cd5b5d7f8,이야 여기 컨셉 맛집이네 ㅋㅋㅋㅋㅋ👍\n\n세심한 부분까지 포인트 넘 잘 살린거 아...,2024년 2월 13일,6772,,C3SB6jMhp6m,우린이제 컨겝이아니라@정말 줌마들이 되가는,2024년 2월 15일,인천,"인천 , #인천"


## 형태소 분류

In [15]:
from konlpy.tag import Okt
okt = Okt()
# 조사, 어미, 구두점 같은 것을 제거
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] in ['Adjective', 'Noun', 'Verb', 'Punctuation', 'KoreanParticle','Number']:
            clean_text.append(word[0])

    return " ".join(clean_text)

In [17]:
df_sns_review['review_clean_okt'] = df_sns_review['review_content'].apply(okt_clean)

In [18]:
train_data['document'] = train_data['document'].astype(str)

In [19]:
train_data['review_clean_okt'] = train_data['document'].apply(okt_clean)

In [20]:
df_sns_review = df_sns_review.query('review_clean_okt != ""')

In [21]:
df_sns_review = df_sns_review.reset_index()
df_sns_review = df_sns_review[['card_content','card_date','card_like','card_watch','review_content','review_date','region','review_clean_okt']]
df_sns_review

,card_content,card_date,card_like,card_watch,review_content,review_date,region,review_clean_okt
0,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,정말 아름다워요,2014년 7월 9일,,정말 아름답다
1,알려지지 않았다고 해서 가치가 없는 것은 아니다. 한 장의 사진으로 사랑 받게 된 ...,2014년 6월 23일,50,,멋진 섬이네요,2014년 6월 24일,,멋지다 서다
2,"유유자적 한가로이 풀을 뜯는 말 뒤편,\n걸려있는 구름마저도 평화로운 성산일출봉\n...",2014년 7월 10일,42,,저 말이 되고싶어요^^,2014년 7월 10일,제주,저 말 되다 ^^
3,"유유자적 한가로이 풀을 뜯는 말 뒤편,\n걸려있는 구름마저도 평화로운 성산일출봉\n...",2014년 7월 10일,42,,Beautiful~,2014년 7월 10일,제주,~
4,울산 명선도의 불게 타오르는 노을과 해무의 장관\n마치 그림을 그려놓은듯한 풍경에 ...,2014년 7월 11일,58,,근데 명산도는 3~4월 물이 빠지면 걸어들갈 수는 있으나 평상시 여행할 수있는 섬은...,2014년 8월 5일,울산,명산 도 3~4월 물이 빠지다 걸다 들다 갈다 수 있다 평상시 여행 하다 수 있다 섬 녜
...,...,...,...,...,...,...,...,...
395242,거창 여행 어떻게 할지 모르겠다고? 😎 여미가 직접 다녀와서 알려주는 [밥상의 여정...,2024년 4월 24일,1288,,여기 가까?,2024년 4월 24일,경남,여기 가깝다 ?
395243,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,미츠바 좋습니당,2024년 4월 26일,서울,미츠바 좋다 습 니당다
395244,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,와 이제 곧 장미공원의 시즌이다,2024년 4월 26일,서울,오다 이제 곧 장미 공원 시즌
395245,"일단 저장!📩 서울에서 데이트, 야장, 피크닉까지 쌈@뽕하게 즐기는 방법😜\n\n서...",2024년 4월 26일,123,,혹시 여기 다 가본 사람 있어? 어디가 젤 좋았는지 추천해주라줘~🙋🏻‍♀️,2024년 4월 26일,서울,혹시 여기 가보다 사람 있다 ? 어디가 젤 좋다 추천 해주다 주다 ~


## 학습, 테스트, 검증 데이터 분류

In [22]:
train_feature = train_data['review_clean_okt']
train_target = train_data['label']

In [23]:
# train, test 나누기
from sklearn.model_selection import train_test_split

feature_train, feature_test, target_train, target_test = train_test_split(train_feature, train_target, test_size=0.6, random_state=42)

## 벡터화하기

In [24]:
feature_train

100923    케빈 스 페이 시 , 줄리 앤 무어 .. 이름 당연하다 연기 말 하다 없다 원작 1...
117033                                      발화점 넘다 못 채 연소 .
179459                                         어이 아리마 셍 ㅎㅎㅎ
177925                                     미군 끄다 평점 높다 대세 ?
138125    결코 평점 9.83 짜다 리 절대 아니다 생각 하다 과 감히 1 점줌 ... 평점 ...
                                ...                        
119879                                      개다 리지 리지 마 변태 년
103694    지루하다 짜증나다 어지럽다 싱겁다 ... 재수 없다 불쾌하다 영화 .. 영화 부르다...
131932                         스크림 같다 영화 만들다 싶다 보다 택 없다 -_-
146867            제작비 탓 하다 빈약하다 각본 . 원작 살리다 채 이유 연결 되다 플롯 .
121958                                         재미없다 .......
Name: review_clean_okt, Length: 80000, dtype: object

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,2))
feature_train = tfidfVectorizer.fit_transform(feature_train)
feature_train.toarray()[:2]
dictionary_list = tfidfVectorizer.get_feature_names_out()
dictionary_list

array(['00', '00 00', '00 감독', ..., '힝힝 정말', '힣히히헤', '힣히히헤 헤헤'],
      dtype=object)

In [26]:
feature_test = tfidfVectorizer.transform(feature_test)

## 긍, 부정 분류 모델 선택하기

### randomforestclassifier

In [83]:
from sklearn.ensemble import RandomForestClassifier
classification_topic = RandomForestClassifier(n_jobs=-1)

In [84]:
classification_topic.fit(feature_train, target_train)

RandomForestClassifier(n_jobs=-1)

In [85]:
target_train_predict = classification_topic.predict(feature_train)

In [86]:
# classification_report로 model 성능 확인
from sklearn.metrics import classification_report
print(classification_report(target_train, target_train_predict))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     39869
           1       0.98      1.00      0.99     40131

    accuracy                           0.99     80000
   macro avg       0.99      0.99      0.99     80000
weighted avg       0.99      0.99      0.99     80000



In [87]:
target_test_predict = classification_topic.predict(feature_test)

In [88]:
from sklearn.metrics import classification_report
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.82      0.81      0.82     60131
           1       0.81      0.82      0.82     59869

    accuracy                           0.82    120000
   macro avg       0.82      0.82      0.82    120000
weighted avg       0.82      0.82      0.82    120000



### logisticRegression

In [89]:
from sklearn import linear_model
logisticRegression=linear_model.LogisticRegression()

In [90]:
logisticRegression.fit(feature_train,target_train)

LogisticRegression()

In [91]:
target_train_predict = logisticRegression.predict(feature_train)

In [92]:
# classification_report로 model 성능 확인
from sklearn.metrics import classification_report
print(classification_report(target_train, target_train_predict))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91     39869
           1       0.92      0.89      0.90     40131

    accuracy                           0.91     80000
   macro avg       0.91      0.91      0.91     80000
weighted avg       0.91      0.91      0.91     80000



In [93]:
target_test_predict = logisticRegression.predict(feature_test)

In [94]:
from sklearn.metrics import classification_report
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.83      0.84      0.83     60131
           1       0.84      0.82      0.83     59869

    accuracy                           0.83    120000
   macro avg       0.83      0.83      0.83    120000
weighted avg       0.83      0.83      0.83    120000



### Naive Bayes 

In [95]:
# from sklearn.naive_bayes import GaussianNB
# nb = GaussianNB()

In [96]:
# nb.fit(feature_train.toarray(),target_train)

In [97]:
# target_train_predict = nb.predict(feature_train.toarray())

In [98]:
# # classification_report로 model 성능 확인
# from sklearn.metrics import classification_report
# print(classification_report(target_train, target_train_predict))

In [99]:
# target_test_predict = nb.predict(feature_test.toarray())

In [100]:
# from sklearn.metrics import classification_report
# print(classification_report(target_test, target_test_predict))

### randomforestclassifier

In [101]:
from sklearn.ensemble import RandomForestClassifier
classification_topic = RandomForestClassifier(n_jobs=-1)

In [102]:
classification_topic.fit(feature_train, target_train)

RandomForestClassifier(n_jobs=-1)

In [103]:
target_train_predict = classification_topic.predict(feature_train)

In [104]:
# classification_report로 model 성능 확인
from sklearn.metrics import classification_report
print(classification_report(target_train, target_train_predict))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     39869
           1       0.98      1.00      0.99     40131

    accuracy                           0.99     80000
   macro avg       0.99      0.99      0.99     80000
weighted avg       0.99      0.99      0.99     80000



In [105]:
target_test_predict = classification_topic.predict(feature_test)

In [106]:
from sklearn.metrics import classification_report
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.82      0.81      0.82     60131
           1       0.81      0.82      0.82     59869

    accuracy                           0.82    120000
   macro avg       0.82      0.82      0.82    120000
weighted avg       0.82      0.82      0.82    120000



### K-Nearest Neighbors

In [107]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [108]:
knn.fit(feature_train, target_train)

KNeighborsClassifier()

In [109]:
target_train_predict = knn.predict(feature_train)

In [110]:
# classification_report로 model 성능 확인
from sklearn.metrics import classification_report
print(classification_report(target_train, target_train_predict))

              precision    recall  f1-score   support

           0       0.73      0.71      0.72     39869
           1       0.72      0.74      0.73     40131

    accuracy                           0.73     80000
   macro avg       0.73      0.73      0.73     80000
weighted avg       0.73      0.73      0.73     80000



In [111]:
target_test_predict = knn.predict(feature_test)

In [112]:
from sklearn.metrics import classification_report
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.60      0.57      0.59     60131
           1       0.59      0.61      0.60     59869

    accuracy                           0.59    120000
   macro avg       0.59      0.59      0.59    120000
weighted avg       0.59      0.59      0.59    120000



### Support Vector Machine

In [27]:
from sklearn import svm
svm_model = svm.SVC()

In [ ]:
svm_model.fit(feature_train, target_train)

In [ ]:
target_train_predict = svm_model.predict(feature_train)

In [ ]:
# classification_report로 model 성능 확인
from sklearn.metrics import classification_report
print(classification_report(target_train, target_train_predict))

In [ ]:
target_test_predict = svm_model.predict(feature_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(target_test, target_test_predict))